In [1]:
## 코드 한번에 합쳐둔거~!

## 구글 드라이브 마운트
from google.colab import drive
import os

drive.mount('/content/drive/')

print('현재 작업 경로 :', os.getcwd())
os.chdir('/content/drive/MyDrive')
print('변경된 작업 경로 :', os.getcwd())

## 함수 import
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# 이미지 데이터를 로드하는 함수
def load_images(directory, label):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith(".png"):
            img_path = os.path.join(directory, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (128, 128))
            images.append(img)
            labels.append(label)
    return images, labels

# 데이터 로드
patients_images, patients_labels = load_images("/content/drive/MyDrive/dataset/patients", label="patients")
controls_images, controls_labels = load_images("/content/drive/MyDrive/dataset/controls", label="controls")

# 데이터 합치기
X = np.array(patients_images + controls_images)
y = np.array(patients_labels + controls_labels)

# 레이블을 숫자로 인코딩
le = LabelEncoder()
y = le.fit_transform(y)

# 데이터를 학습 세트와 테스트 세트로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=36)

# 데이터 전처리 및 LSTM layer에 맞게 reshape
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

X_train = X_train.reshape((X_train.shape[0], 128, 128))
X_test = X_test.reshape((X_test.shape[0], 128, 128))

# 모델 구축
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1:]), activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# 모델 요약
model.summary()

# 모델 훈련
model.fit(X_train, y_train, epochs=100, batch_size=10)

# 모델 평가
evaluation_result = model.evaluate(X_test, y_test)

loss = evaluation_result[0]
accuracy = evaluation_result[1]
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Mounted at /content/drive/
현재 작업 경로 : /content
변경된 작업 경로 : /content/drive/MyDrive
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128, 50)           35800     
                                                                 
 dropout (Dropout)           (None, 128, 50)           0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 56051 (218.95 KB)
Trainable params: 56051 (218.95 KB)
Non-trainable params: 0 (0.00 Byte)
__

In [2]:
# 모델 평가
model.evaluate(X_test, y_test)

5/5 [==============================] - 0s 19ms/step - loss: 0.6932 - accuracy: 0.5000


[0.6931564211845398, 0.5]

In [3]:
#Building the Decision Tree Model on our dataset
from sklearn.tree import DecisionTreeRegressor

predicted = model.predict(X_test)

5/5 [==============================] - 0s 18ms/step


In [5]:
predicted

array([[0.5007771 ],
       [0.500525  ],
       [0.5005471 ],
       [0.50053304],
       [0.5007762 ],
       [0.5006388 ],
       [0.50050277],
       [0.50049794],
       [0.50050485],
       [0.5005391 ],
       [0.500741  ],
       [0.50071716],
       [0.5007751 ],
       [0.5005421 ],
       [0.5007061 ],
       [0.500725  ],
       [0.5007712 ],
       [0.5007149 ],
       [0.5005887 ],
       [0.5007751 ],
       [0.50065935],
       [0.5005589 ],
       [0.50073045],
       [0.5005589 ],
       [0.5005264 ],
       [0.5007286 ],
       [0.50072813],
       [0.5007368 ],
       [0.5007226 ],
       [0.500555  ],
       [0.5007751 ],
       [0.50077707],
       [0.5007363 ],
       [0.50072676],
       [0.5004969 ],
       [0.5006159 ],
       [0.5005395 ],
       [0.50073844],
       [0.50073   ],
       [0.5007192 ],
       [0.50060457],
       [0.50053483],
       [0.5007302 ],
       [0.5007762 ],
       [0.5007205 ],
       [0.5007389 ],
       [0.50077575],
       [0.500

In [4]:
# 각 행에서 더 큰 값의 인덱스를 추출하여 1 또는 0으로 구성된 리스트 생성
predicted_classes = (predicted[:, 1] > predicted[:, 0]).astype(int)

IndexError: ignored

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predicted_classes))

In [8]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# plot the confusion matrix
mat = confusion_matrix(y_test, predicted)
sns.heatmap(mat.T, square = True, annot=True, xticklabels = sorted(set(y_test)), yticklabels = sorted(set(y_test)), fmt = "d")
plt.xlabel("true labels")
plt.ylabel("predicted label")
plt.show()

ValueError: ignored